In [1]:
# Realizando os primeiros imports

import pandas as pd
import numpy as np

In [2]:
# Carregando os dados

data_train = pd.read_csv("train.csv")
data_submiss = pd.read_csv("test.csv")

# Vizualisando os DataFrames

In [3]:
data_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
data_submiss.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 208.2+ KB


In [6]:
data_submiss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 63.8+ KB


# PREPARANDO DADOS

In [7]:
#import nltk
import re
import string
import unicodedata
from tqdm import tqdm

In [8]:
def clear_text(text):
    # REMOÇÃO DE LINKS
    text = re.sub(r'http\S+','',text).replace('  ',' ').lower()

    return text

In [9]:
data_train['clear_text'] = tqdm(data_train['text'].map(clear_text))
data_submiss['clear_text'] = tqdm(data_submiss['text'].map(clear_text))

100%|██████████| 3263/3263 [00:00<00:00, 296690.02it/s]


# BUSCA PELO MELHOR MODELO

In [10]:
from search_best_model import BestModel

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [11]:
dataEntry = pd.DataFrame({'values': data_train.clear_text, 'target':data_train.target})

dataEntry.head()

,values,target
0,our deeds are the reason of this #earthquake m...,1
1,forest fire near la ronge sask. canada,1
2,all residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,just got sent this photo from ruby #alaska as ...,1


In [12]:
dict_result : dict = BestModel.bestClassifierSparse(dataEntry)

100%|██████████| 30/30 [00:09<00:00,  9.57s/trial, best loss: 0.196078431372549]


In [29]:
print(f"""
Accuracy: {dict_result['score']}

Best Model: {dict_result['best_model']['learner']}

Best Pre-process: {dict_result['best_model']['preprocs'][0]}
""")


Accuracy: 0.7950656585754079

Best Model: SGDClassifier(alpha=0.0028535230324935874, class_weight='balanced',
              eta0=0.00028120678640011987, l1_ratio=0.6155089860686642,
              learning_rate='invscaling', loss='squared_loss',
              max_iter=44964398.0, n_jobs=1, power_t=0.08296412658986185,
              random_state=4, tol=6.945405464915281e-05, verbose=False)

Best Pre-process: TfidfVectorizer(binary=True, ngram_range=(1, 3), norm=None, smooth_idf=False,
                use_idf=False)



# Definindo modelo e processamento

In [14]:
from sklearn.linear_model import SGDClassifier, RidgeClassifier

from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score

In [38]:
# Modelo
clf = SGDClassifier(alpha=0.0028535230324935874, class_weight='balanced',
              eta0=0.00028120678640011987, l1_ratio=0.6155089860686642,
              learning_rate='invscaling', loss='squared_loss',
              max_iter=44964398.0, n_jobs=1, power_t=0.08296412658986185,
              random_state=4, tol=6.945405464915281e-05, verbose=False)

# Processamento
pre = TfidfVectorizer(binary=True, ngram_range=(1, 3), norm=None, smooth_idf=False,
                use_idf=False)

In [39]:
# Validação de modelo

X_train, X_test, y_train, y_test = train_test_split(data_train['clear_text'], data_train['target'], test_size=0.33, random_state=1)

features_train = pre.fit_transform(X_train)
features_test = pre.transform(X_test)

clf.fit(features_train, y_train)

predict_test = clf.predict(features_test)

print(f"{accuracy_score(y_test, predict_test)}")

0.7950656585754079


In [40]:
# Realizando treinamento com conjunto completo de dados

features_train = pre.fit_transform(data_train.clear_text)

features_submiss = pre.transform(data_submiss.clear_text)

clf.fit(features_train, data_train.target)

SGDClassifier(alpha=0.0028535230324935874, class_weight='balanced',
              eta0=0.00028120678640011987, l1_ratio=0.6155089860686642,
              learning_rate='invscaling', loss='squared_loss',
              max_iter=44964398.0, n_jobs=1, power_t=0.08296412658986185,
              random_state=4, tol=6.945405464915281e-05, verbose=False)

In [41]:
# Realizando a previsão

pred = clf.predict(features_submiss)

# Gerando CSV para submissao

In [43]:
last_result = pd.DataFrame({'id':data_submiss.id, 'target':pred})
last_result.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [44]:
last_result.to_csv('second_submission.csv', index=False)